In [18]:
from sklearn.datasets import fetch_openml
df = fetch_openml("parity5_plus_5", version=1).frame

In [19]:
df

,Bit_1,Bit_2,Bit_3,Bit_4,Bit_5,Bit_6,Bit_7,Bit_8,Bit_9,Bit_10,class
0,1,1,0,0,1,0,0,0,0,1,1
1,1,0,0,0,0,1,1,1,1,0,0
2,0,0,1,0,1,1,1,1,1,1,1
3,0,1,0,1,0,0,0,1,0,0,1
4,0,0,0,0,0,0,0,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...
1119,0,0,1,0,1,1,0,1,1,0,1
1120,0,1,0,0,1,0,0,0,1,1,1
1121,0,1,0,0,1,1,0,0,1,0,0
1122,1,1,0,0,1,1,1,0,1,0,0


In [20]:
from scripts.transformer_prediction_interface import TabPFNClassifier
tabpfn = TabPFNClassifier(device="cpu", model_string="tabpfn__emsize_512_nlayers_12_steps_2048_bs_32ada_lr_0.0001_1_gpu_07_24_2023_01_43_33", epoch="1650", N_ensemble_configurations=32)

In [21]:
from sklearn.model_selection import cross_validate

In [22]:
df_int = df.astype(int)
X, y = df_int.drop(columns="class"), df_int['class']

In [23]:
X = X[['Bit_2', 'Bit_3', 'Bit_4', 'Bit_6', 'Bit_8']]

In [6]:
tabpfn.fit(X[::2], y[::2]).score(X, y)

Using a Transformer with 25.83 M parameters
Using cpu device


0.5044483985765125

In [24]:
cross_validate(tabpfn, X, y)

Using a Transformer with 25.83 M parameters
Using cpu device


{'fit_time': array([0.76405501, 0.00290895, 0.00401354, 0.00190806, 0.00192833]),
 'score_time': array([14.76918888, 14.59478641, 14.13186193, 13.20889926, 10.89754367]),
 'test_score': array([0.50666667, 0.50666667, 0.50222222, 0.50222222, 0.50446429])}

In [10]:
cross_validate(tabpfn, X, y, scoring="roc_auc")

{'fit_time': array([0.00216126, 0.00228119, 0.00386071, 0.00199556, 0.00214863]),
 'score_time': array([15.33299661, 13.43223691, 17.92081046, 13.73480487, 15.28722167]),
 'test_score': array([0.38391023, 0.35277383, 0.26438053, 0.34347345, 0.28238858])}

In [29]:
from tabpfn.transformer_make_model import ShiftClassifier, EnsembleMeta, ForwardMLPModel
device = "cpu"
mothernet_21_46_25_3940_ensemble3 = EnsembleMeta(ForwardMLPModel(path="mn_d2048_H4096_L2_W32_P512_1_gpu_warm_08_25_2023_21_46_25_epoch_3940_no_optimizer.pickle", device=device), n_estimators=3)


In [31]:
cross_validate(mothernet_21_46_25_3940_ensemble3, np.array(X), np.array(y), scoring="roc_auc")

{'fit_time': array([4.98182416, 5.80485392, 0.93599319, 3.49394512, 8.14269161]),
 'score_time': array([0.07250881, 0.13013983, 0.09660625, 0.19630408, 0.23274612]),
 'test_score': array([0.47273589, 0.48087561, 0.43418142, 0.50426675, 0.42796779])}

Bad pipe message: %s [b"~\x1b,v\xe7\x96GB\xe2b\xdb\x00\xbd3{\x17y7\x00\x00|\xc0,\xc00\x00\xa3\x00\x9f\xcc\xa9\xcc\xa8\xcc\xaa\xc0\xaf\xc0\xad\xc0\xa3\xc0\x9f\xc0]\xc0a\xc0W\xc0S\xc0+\xc0/\x00\xa2\x00\x9e\xc0\xae\xc0\xac\xc0\xa2\xc0\x9e\xc0\\\xc0`\xc0V\xc0R\xc0$\xc0(\x00k\x00j\xc0#\xc0'\x00g\x00@\xc0\n\xc0\x14\x009\x008\xc0\t\xc0\x13\x003\x002\x00\x9d\xc0\xa1\xc0\x9d\xc0Q\x00\x9c\xc0\xa0\xc0\x9c\xc0P\x00=\x00<\x005\x00/\x00\x9a\x00\x99\xc0\x07\xc0\x11\x00\x96\x00\x05\x00\xff\x01\x00\x00j\x00\x00\x00\x0e\x00\x0c\x00\x00\t127.0.0.1\x00\x0b\x00\x04\x03\x00\x01\x02\x00\n\x00\x0c\x00\n\x00\x1d\x00\x17\x00\x1e\x00\x19\x00\x18\x00#\x00\x00\x00\x16\x00\x00\x00\x17\x00\x00\x00\r\x000\x00.\x04\x03\x05\x03\x06\x03\x08\x07\x08\x08\x08\t\x08\n\x08\x0b\x08\x04", b'\x08\x06\x04\x01\x05\x01\x06', b'', b'\x03\x03']
Bad pipe message: %s [b'\xa8\x155\xa9H\xbb\xf3\xf0\x1a@\x1e\xf3\x1aD3\x02\x17\x01 \x9fsG\x8b<}\xc7\x81\xf7\xf79L\x11']
Bad pipe message: %s [b"Gp~\xaf\xe2\x06\xce\x05k\xbb\xedm4\xe4\xad\x93v`

In [25]:
from xgboost import XGBClassifier
cross_validate(XGBClassifier(), X, y, scoring="roc_auc")

{'fit_time': array([ 0.31549406,  0.08988118,  0.79190493, 13.34164381,  0.19409513]),
 'score_time': array([0.00369096, 0.00952458, 0.00296426, 0.02641153, 0.05660844]),
 'test_score': array([1., 1., 1., 1., 1.])}

In [20]:
cross_validate(XGBClassifier(), X, y)

{'fit_time': array([3.87103653, 0.18429685, 0.11258483, 6.81403637, 2.59869337]),
 'score_time': array([0.00282526, 0.01548386, 0.0343821 , 0.00302696, 0.00275564]),
 'test_score': array([1.        , 1.        , 1.        , 0.99555556, 1.        ])}

In [6]:
from tabpfn.scripts.distill_mlp import TorchMLP

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y)

NameError: name 'X' is not defined

In [22]:
import numpy as np
mlp = TorchMLP(verbose=10, n_epochs=1000, n_layers=1).fit(np.array(X_train), np.array(y_train))

epoch: 0  loss: 0.703014  [  843/  843]
epoch: 10  loss: 0.693446  [  843/  843]
epoch: 20  loss: 0.690363  [  843/  843]
epoch: 30  loss: 0.688065  [  843/  843]
epoch: 40  loss: 0.685750  [  843/  843]
epoch: 50  loss: 0.682968  [  843/  843]
epoch: 60  loss: 0.679543  [  843/  843]
epoch: 70  loss: 0.675316  [  843/  843]
epoch: 80  loss: 0.670105  [  843/  843]
epoch: 90  loss: 0.663819  [  843/  843]
epoch: 100  loss: 0.656415  [  843/  843]
epoch: 110  loss: 0.647888  [  843/  843]
epoch: 120  loss: 0.638151  [  843/  843]
epoch: 130  loss: 0.627276  [  843/  843]
epoch: 140  loss: 0.615164  [  843/  843]
epoch: 150  loss: 0.601554  [  843/  843]
epoch: 160  loss: 0.586333  [  843/  843]
epoch: 170  loss: 0.569679  [  843/  843]
epoch: 180  loss: 0.551759  [  843/  843]
epoch: 190  loss: 0.533003  [  843/  843]
epoch: 200  loss: 0.513910  [  843/  843]
epoch: 210  loss: 0.495036  [  843/  843]
epoch: 220  loss: 0.475654  [  843/  843]
epoch: 230  loss: 0.456573  [  843/  843]
epo

In [23]:
mlp.score(np.array(X_train), np.array(y_train))

1.0

In [24]:
mlp.score(np.array(X_test), np.array(y_test))

1.0

In [25]:
tabpfn_old = TabPFNClassifier(device="cpu")

In [26]:
cross_validate(tabpfn_old, X, y, scoring="roc_auc")

Using a Transformer with 25.82 M parameters
Using cpu device
Multiple models in memory. This might lead to memory issues. Consider calling remove_models_from_memory()


{'fit_time': array([0.21992421, 0.00204849, 0.00212264, 0.0020833 , 0.00205779]),
 'score_time': array([5.16884279, 6.47356462, 8.42200708, 4.17055202, 2.94250083]),
 'test_score': array([0.36526   , 0.42776988, 0.42240834, 0.45314475, 0.43402695])}

In [27]:
tabpfn_old.base__path

PosixPath('/home/azureuser/TabPFN/tabpfn')

In [29]:
tabpfn_old.model_string

'download'

In [31]:
tabpfn_old.fit(X[::4], y[::4])

TabPFNClassifier()

In [33]:
tabpfn_old.model

TransformerModel(
  (transformer_encoder): TransformerEncoderDiffInit(
    (layers): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=512, out_features=512, bias=True)
        )
        (linear1): Linear(in_features=512, out_features=1024, bias=True)
        (dropout): Dropout(p=0.0, inplace=False)
        (linear2): Linear(in_features=1024, out_features=512, bias=True)
        (norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.0, inplace=False)
        (dropout2): Dropout(p=0.0, inplace=False)
      )
      (1): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=512, out_features=512, bias=True)
        )
        (linear1): Linear(in_features=512, out_features=1024, bias=True)
        (dr

In [41]:
import xgboost
cross_validate(xgboost.XGBClassifier(), data, labels)

{'fit_time': array([ 2.98640275, 17.15443826,  7.28647876,  8.05638838,  2.61188293]),
 'score_time': array([0.02973866, 0.02244353, 0.00227427, 0.07001352, 0.00319123]),
 'test_score': array([0.  , 0.1 , 0.  , 0.05, 0.  ])}

In [60]:
cross_validate(xgboost.XGBClassifier(), data_onehot, labels)

{'fit_time': array([11.27976203,  1.46875477,  4.22466969,  7.14353752, 16.1927278 ]),
 'score_time': array([0.01101518, 0.00264144, 0.00274181, 0.00294971, 0.00236106]),
 'test_score': array([0.15, 0.1 , 0.  , 0.  , 0.1 ])}

In [37]:
from sklearn.ensemble import RandomForestClassifier
cross_validate(RandomForestClassifier(), data, labels)

{'fit_time': array([0.08397508, 0.07852745, 0.07594156, 0.08024979, 0.0786345 ]),
 'score_time': array([0.00678587, 0.00607204, 0.00601339, 0.00677276, 0.01009989]),
 'test_score': array([0.05, 0.  , 0.  , 0.  , 0.05])}

In [35]:
cross_validate(tabpfn_old, data, labels)

{'fit_time': array([0.00084782, 0.00046349, 0.00047493, 0.00044036, 0.00042224]),
 'score_time': array([0.20250225, 0.17874169, 1.46880245, 3.54743743, 0.93717194]),
 'test_score': array([0.45, 0.7 , 0.4 , 0.35, 0.4 ])}

In [59]:
cross_validate(tabpfn_old, data_onehot, labels)

{'fit_time': array([0.0008018 , 0.00042033, 0.00042772, 0.00043058, 0.00042224]),
 'score_time': array([1.77699208, 0.3342917 , 0.3345921 , 0.62064624, 0.48883986]),
 'test_score': array([0.2 , 0.2 , 0.15, 0.25, 0.25])}

In [1]:
import numpy as np
x, y = np.c_[np.meshgrid(np.arange(10), np.arange(10))]
x, y = x.ravel(), y.ravel()

z = (x + y) % 7
z

labels = z
data = np.c_[x, y]

In [2]:
labels.shape

(100,)

In [3]:
from sklearn.preprocessing import OneHotEncoder
data_onehot = OneHotEncoder(sparse=False).fit_transform(data)

In [4]:
data_onehot

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [5]:
data_onehot.shape

(100, 20)

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, labels)

In [7]:
X_train_onehot, X_test_onehot, y_train, y_test = train_test_split(data_onehot, labels)

In [8]:
from tabpfn.scripts.distill_mlp import TorchMLP
mlp = TorchMLP(verbose=10, n_epochs=10000, n_layers=4, dropout_rate=0.1).fit(np.array(X_train), np.array(y_train))

/anaconda/envs/tabpfn/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


epoch: 0  loss: 1.946566  [   75/   75]
epoch: 10  loss: 1.885500  [   75/   75]
epoch: 20  loss: 1.795502  [   75/   75]
epoch: 30  loss: 1.799053  [   75/   75]
epoch: 40  loss: 1.726891  [   75/   75]
epoch: 50  loss: 1.701384  [   75/   75]
epoch: 60  loss: 1.673452  [   75/   75]
epoch: 70  loss: 1.623352  [   75/   75]
epoch: 80  loss: 1.627737  [   75/   75]
epoch: 90  loss: 1.615990  [   75/   75]
epoch: 100  loss: 1.560700  [   75/   75]
epoch: 110  loss: 1.535896  [   75/   75]
epoch: 120  loss: 1.535925  [   75/   75]
epoch: 130  loss: 1.457234  [   75/   75]
epoch: 140  loss: 1.436597  [   75/   75]
epoch: 150  loss: 1.446136  [   75/   75]
epoch: 160  loss: 1.443652  [   75/   75]
epoch: 170  loss: 1.435370  [   75/   75]
epoch: 180  loss: 1.365004  [   75/   75]
epoch: 190  loss: 1.334877  [   75/   75]
epoch: 200  loss: 1.351780  [   75/   75]
epoch: 210  loss: 1.324790  [   75/   75]
epoch: 220  loss: 1.247710  [   75/   75]
epoch: 230  loss: 1.320609  [   75/   75]
epo

In [9]:
mlp.score(X_train, y_train)

0.9733333333333334

In [10]:
mlp.score(X_test, y_test)

0.04

In [17]:
from tabpfn.scripts.distill_mlp import TorchMLP

mlp = TorchMLP(verbose=15, n_epochs=10000, n_layers=4, learning_rate=0.0001, dropout_rate=0.2, weight_decay=0.05, validation_data=(X_test_onehot, y_test)).fit(X_train_onehot, y_train)

epoch: 0  loss: 1.947401  [   75/   75]
Validation loss: 1.959167
validation score: 0.080000
epoch: 10  loss: 1.944291  [   75/   75]
Validation loss: 1.960477
validation score: 0.080000
epoch: 20  loss: 1.945593  [   75/   75]
Validation loss: 1.960677
validation score: 0.000000
epoch: 30  loss: 1.943575  [   75/   75]
Validation loss: 1.963030
validation score: 0.080000
epoch: 40  loss: 1.943719  [   75/   75]
Validation loss: 1.961795
validation score: 0.080000
epoch: 50  loss: 1.939376  [   75/   75]
Validation loss: 1.963729
validation score: 0.080000
epoch: 60  loss: 1.941168  [   75/   75]
Validation loss: 1.962929
validation score: 0.080000
epoch: 70  loss: 1.939802  [   75/   75]
Validation loss: 1.965259
validation score: 0.080000
epoch: 80  loss: 1.940441  [   75/   75]
Validation loss: 1.966516
validation score: 0.080000
epoch: 90  loss: 1.937190  [   75/   75]
Validation loss: 1.968368
validation score: 0.080000
epoch: 100  loss: 1.932559  [   75/   75]
Validation loss: 1.

In [10]:
mlp.score(X_train_onehot, y_train)

1.0

In [11]:
mlp.score(X_test_onehot, y_test)

0.28

In [13]:
from sklearn.datasets import load_iris
iris = load_iris()
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target)

In [14]:
mlp = TorchMLP(verbose=15, n_epochs=100, n_layers=2, dropout_rate=0.0, validation_data=(X_test, y_test)).fit(X_train, y_train)

epoch: 0  loss: 1.036796  [  112/  112]
> /home/azureuser/TabPFN/tabpfn/scripts/distill_mlp.py(96)fit_from_dataloader()
     94                             X_val, y_val = self.validation_data
     95                             import pdb; pdb.set_trace()
---> 96                             pred = model(torch.tensor(X_val, device=self.device, dtype=torch.float32))
     97                             loss = loss_fn(pred, torch.tensor(y_val, device=self.device))
     98                             print(f"Validation loss: {loss.item():>7f}")



ipdb>  c


Validation loss: 0.978168
epoch: 10  loss: 0.635766  [  112/  112]
> /home/azureuser/TabPFN/tabpfn/scripts/distill_mlp.py(95)fit_from_dataloader()
     93                         with torch.no_grad():
     94                             X_val, y_val = self.validation_data
---> 95                             import pdb; pdb.set_trace()
     96                             pred = model(torch.tensor(X_val, device=self.device, dtype=torch.float32))
     97                             loss = loss_fn(pred, torch.tensor(y_val, device=self.device))



ipdb>  c


Validation loss: 0.590949
epoch: 20  loss: 0.437957  [  112/  112]
> /home/azureuser/TabPFN/tabpfn/scripts/distill_mlp.py(96)fit_from_dataloader()
     94                             X_val, y_val = self.validation_data
     95                             import pdb; pdb.set_trace()
---> 96                             pred = model(torch.tensor(X_val, device=self.device, dtype=torch.float32))
     97                             loss = loss_fn(pred, torch.tensor(y_val, device=self.device))
     98                             print(f"Validation loss: {loss.item():>7f}")



...

Validation loss: 0.108564
epoch: 70  loss: 0.100775  [  112/  112]
> /home/azureuser/TabPFN/tabpfn/scripts/distill_mlp.py(95)fit_from_dataloader()
     93                         with torch.no_grad():
     94                             X_val, y_val = self.validation_data
---> 95                             import pdb; pdb.set_trace()
     96                             pred = model(torch.tensor(X_val, device=self.device, dtype=torch.float32))
     97                             loss = loss_fn(pred, torch.tensor(y_val, device=self.device))



ipdb>  c


Validation loss: 0.090866
epoch: 80  loss: 0.088809  [  112/  112]
> /home/azureuser/TabPFN/tabpfn/scripts/distill_mlp.py(96)fit_from_dataloader()
     94                             X_val, y_val = self.validation_data
     95                             import pdb; pdb.set_trace()
---> 96                             pred = model(torch.tensor(X_val, device=self.device, dtype=torch.float32))
     97                             loss = loss_fn(pred, torch.tensor(y_val, device=self.device))
     98                             print(f"Validation loss: {loss.item():>7f}")



ipdb>  exit


BdbQuit: 